# Demonstration of low-level SQL & PreCalc functions within ChebyChecker
 - This is likely to mainly be of use while developing code 
 - MJP

In [4]:
import time
import numpy as np
import scipy.stats as stats
import math
import random
from collections import defaultdict
import os
import sys
from collections import Counter
import glob 
import warnings
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import importlib
from astropy.time import Time
import pickle


# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------

# orbit_cheby & nbody_reader imports 
#sys.path.append( "/Users/matthewjohnpayne/Envs/orbit_cheby/orbit_cheby/" )
from orbit_cheby import orbit_cheby
from orbit_cheby import nbody_reader
importlib.reload(orbit_cheby)
importlib.reload(nbody_reader)

# sql import(s) 
from orbit_cheby import obs_pos
importlib.reload(obs_pos)

# sql import(s) 
from orbit_cheby import sql
importlib.reload(sql)

# PreCalc import(s) 
from orbit_cheby import precalc
importlib.reload(precalc)


<module 'orbit_cheby.precalc' from '/Users/matthewjohnpayne/opt/anaconda3/lib/python3.7/site-packages/orbit_cheby/precalc.py'>

# Background
 - I expect that the "orbit_cheby" module exists and has functionality to take the results of an n-body integration and convert them into a dictionary of chebyshev coefficients (with seprate coefficient sets for each "sector" of time)  - There is a single dictionary for each object


# --------- Demo-Data Functions ---------------------

 - These functions create fake data for the sake of demonstrating functionalities

In [5]:
def create_sample_cheby_dict():
    ''' convenience function to make demo MSC-list '''
    importlib.reload(nbody_reader)
    importlib.reload(orbit_cheby)

    # Pre-populate numpy array by reading from demo file ...
    text_filepath = os.path.join(os.path.dirname(os.getcwd() ), 'dev_data', '2022AA_demo.txt')
    assert os.path.isfile(text_filepath)
    name, times, states  = nbody_reader.parse_nbody_txt( text_filepath )


    # Create MSCs & return 
    return orbit_cheby.MSC_Loader(FROM_ARRAY = True , 
                            primary_unpacked_provisional_designations = name, 
                            times_TDB = times, 
                            statearray = states).MSCs


In [6]:
MSCs = create_sample_cheby_dict()

print( 'Returned variable is of type %r , and length %r  ' % (type(MSCs),len(MSCs) ) )
M = MSCs[0]
print( 'Entry in list is of type', type(M) )
print ( 'Number of sectors = ' , len(M.sector_coeffs) )
print ( 'Shape of sector[0] = ' , M.sector_coeffs[0].shape )
print ( 'Shape of sector[1] = ' , M.sector_coeffs[0].shape )
print ( 'Shape of sector[517] = ' , M.sector_coeffs[0].shape )
print ( 'Range of validity : ', M.get_valid_range_of_dates() ) 
print ( 'primary_unpacked_provisional_designation : ', M.primary_unpacked_provisional_designation ) 
del M 


INIT MSC_Loader...
Returned variable is of type <class 'list'> , and length 1  
Entry in list is of type <class 'orbit_cheby.orbit_cheby.MSC'>
Number of sectors =  623
Shape of sector[0] =  (18, 27)
Shape of sector[1] =  (18, 27)
Shape of sector[517] =  (18, 27)
Range of validity :  (2440000, 2459999)
primary_unpacked_provisional_designation :  2022 AA


# --------- SQL ---------------------
 - I was doing a lot of reinventing the wheel 
 - I am going to try to radically simplify things and just use a/an SQLITE database to store the required data
 
Required Structure for db ... 
 - Cheby Coefficient table organized with one-row-per-object, and with individual fields (columns) for each 32-day time-sector
 - I assume that the contents of each "cell" will have to be a "blob" containing all required coeffs for all required coords/covars
 - There should also be a column (or separate table?) recording the designation-string

In [14]:
# --- Demonstrate some convenience code to make database with required tables ---
# -------------------------------------------------------------------------------

def convenience_func_create_db_and_tables():
    importlib.reload(sql)
    
    # In order to save data, we require sql-db to exist, so let's set that up...
    # Force deletion then creation of db...
    if os.path.isfile( sql.fetch_db_filepath() ):
        os.remove(sql.fetch_db_filepath())
    conn = sql.create_connection( sql.fetch_db_filepath() )
    cur  = conn.cursor()
    
    # Test creation of db
    assert os.path.isfile( os.path.join( sql.fetch_db_filepath() ) ), 'no db'

    # Create required table(s)
    sql.create_object_coefficients_table(conn)
    sql.create_objects_by_jdhp_table(conn)
    sql.create_object_desig_table(conn)
    

    # Double-check that this worked by getting the count of tables with the name
    # - if the count is 1, then table exists
    cur.execute('SELECT name from sqlite_master WHERE type = "table" AND name = "objects_by_jdhp"')
    assert len(cur.fetchone()) == 1 , 'jdhp table does not exist'
    cur.execute('SELECT name from sqlite_master WHERE type = "table" AND name = "object_coefficients"')
    assert len(cur.fetchone()) == 1 , 'coeff table does not exist'
    cur.execute('SELECT name from sqlite_master WHERE type = "table" AND name = "object_desig"')
    assert len(cur.fetchone()) == 1 , 'coeff table does not exist'

    return conn

# Create db & tables
conn = convenience_func_create_db_and_tables()
# Repeat check that db exists (other checks done within *convenience_func_create_db_and_tables()* )
os.path.isfile( os.path.join( sql.fetch_db_filepath() ) )

 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***


True

# Demonstrate code to "upsert" primary_unpacked_provisional_designation into checker sqlite db 


In [56]:
importlib.reload(sql)

# (0) Create demo MSC(s) & create empty db
MSCs = create_sample_cheby_dict()
#conn = convenience_func_create_db_and_tables()

# (1) Low level direct call using sql
result = sql.insert_desig(conn ,MSCs[0].primary_unpacked_provisional_designation )
print('result = object_id = ' , result )

# (2) Query functionality ...
result = sql.query_number_by_desig(conn, MSCs[0].primary_unpacked_provisional_designation)
print('result = object_id = ' , result )


INIT MSC_Loader...
result = object_id =  3
result = object_id =  3


# Demonstrate code to "upsert" MSC into checker sqlite db 

In [15]:
# --- Demonstrate code to "upsert" MSC into mpchecker2 sqlite db ----------------
# - N.B. in practice a "higher level" function precalc.upsert() will be used ----
#        See cells below for example of precalc.upsert() function ...
# -------------------------------------------------------------------------------
importlib.reload(sql)

# (0) Create demo MSC(s) & create empty db
MSCs = create_sample_cheby_dict()
conn = convenience_func_create_db_and_tables()

# (1) Low level direct call using sql
sql.upsert_MSC(conn ,MSCs[0] )

# (2) Demonstrate that there is something in the database by querying it ... 
cur = conn.cursor()
cur.execute("SELECT * FROM object_coefficients WHERE primary_unpacked_provisional_designation=?", ( MSCs[0].primary_unpacked_provisional_designation, ))

print('Query results ... : name ') 
for row in cur.fetchall(): 
    print(row[1] )#, pickle.loads( row[-1] ).shape  ) 


names = [description[0] for description in cur.description]
print('column names = ' , names )


INIT MSC_Loader...
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
Query results ... : name 
2022 AA
column names =  ['id', 'primary_unpacked_provisional_designation', 'sector_0_2440000', 'sector_1_2440032', 'sector_2_2440064', 'sector_3_2440096', 'sector_4_2440128', 'sector_5_2440160', 'sector_6_2440192', 'sector_7_2440224', 'sector_8_2440256', 'sector_9_2440288', 'sector_10_2

# --- Demonstrate code to query checker db for object-coefficients

In [16]:
# Set up connection to db to enable subseqjuent low-level query  
importlib.reload(sql)
conn = sql.create_connection( sql.fetch_db_filepath() )

# Execute the query (searching against primary_unpacked_provisional_designation == '2022 AA' )
print('Input desig ... ', MSCs[0].primary_unpacked_provisional_designation)
result = sql.query_object_coefficients(conn , MSCs[0].primary_unpacked_provisional_designation )

# Examine the characteristics of the output 
print('Characteristics of query results ...' , type(result) , len(result) )
key0 = list(result.keys())[0] ; print('\t'*2, ' key0 ... ', key0, result[key0].shape )

# Compare the output to the input to verify is the same ... 
if np.all(MSCs[0].sector_coeffs[ int(key0.split("_")[1]) ] == result[key0] ) : print( 'In == Out !!')

 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
Input desig ...  2022 AA
Characteristics of query results ... <class 'dict'> 623
		  key0 ...  sector_0_2440000 (18, 27)
In == Out !!


### --- query checker db for object-coefficients over a limited subset of sectors

In [17]:
# Set up connection to db to enable subseqjuent low-level query  
importlib.reload(sql)
conn = sql.create_connection( sql.fetch_db_filepath() )

# Execute the query (searching against primary_unpacked_provisional_designation == '2022 AA' )
print('Input desig ... ', MSCs[0].primary_unpacked_provisional_designation)
result = sql.query_object_coefficients(conn , 
                                       MSCs[0].primary_unpacked_provisional_designation , 
                                      sector_numbers = range(0,4))
# Examine the characteristics of the output 
print('Characteristics of query results ...' , type(result) , len(result) )
key0 = list(result.keys())[0] ; print('\t'*2, ' key0 ... ', key0, result[key0].shape )

# Compare the output to the input to verify is the same ... 
if np.all(MSCs[0].sector_coeffs[ int(key0.split("_")[1]) ] == result[key0] ) : print( 'In == Out !!')

 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
Input desig ...  2022 AA
Characteristics of query results ... <class 'dict'> 4
		  key0 ...  sector_0_2440000 (18, 27)
In == Out !!


# Demonstrate code to "upsert" HealPix into checker sqlite db 

In [11]:
%%time
importlib.reload(obs_pos)
#
# *** *** *** *** *** *** *** THIS TAKES ~200 SECONDS TO RUN *** *** *** *** *** *** ***
#
# Create a sample of observatory-positions as inputs 
# - Note that the NBody code uses HELIOCENTRIC EQUATORIAL coords
# - So the Cheby's need to work-in/evaluate-to HELIOCENTRIC EQUATORIAL coords
# - Hence the observatory-positions will need to be in HELIOCENTRIC EQUATORIAL coords
# - We will use the ObsPos class to get this data (see '' demo notebook for more details)
JDlist = orbit_cheby.Base().JDlist 
observatoryXYZ = np.array( [obs_pos.ObsPos().get_heliocentric_equatorial_xyz(jd , obsCode='500') for jd in JDlist] ).T
print(len(JDlist) , JDlist[:3], '...',JDlist[-3:])
print(len(observatoryXYZ) , observatoryXYZ[:3], '...',observatoryXYZ[-3:])

20000 [2440000 2440001 2440002] ... [2459997 2459998 2459999]
3 [[-0.46118508 -0.44608068 -0.43084806 ... -0.87534198 -0.88349422
  -0.89137455]
 [-0.82722792 -0.83436097 -0.84125573 ...  0.42224568  0.4081433
   0.39391627]
 [-0.35872031 -0.36181405 -0.36480441 ...  0.18304707  0.17693332
   0.17076544]] ... [[-0.46118508 -0.44608068 -0.43084806 ... -0.87534198 -0.88349422
  -0.89137455]
 [-0.82722792 -0.83436097 -0.84125573 ...  0.42224568  0.4081433
   0.39391627]
 [-0.35872031 -0.36181405 -0.36480441 ...  0.18304707  0.17693332
   0.17076544]]
CPU times: user 3min 44s, sys: 4.5 s, total: 3min 49s
Wall time: 3min 52s


In [57]:
%%time
# Now explicitly evaluate the HP functionality to get the appropriate HP for each date 
# - See "Demonstrate_Orbital_Chebyshev_Functionality" demo for more details 
n = 15000
HPlist= MSCs[0].generate_HP( JDlist[:n] , 
                observatoryXYZ[:,:n], 
                APPROX = True )

CPU times: user 74.5 ms, sys: 10.3 ms, total: 84.8 ms
Wall time: 82.8 ms


In [58]:
importlib.reload(sql)
# Now the code to do the upsert ...
sql.upsert_HP(conn, MSCs[0].primary_unpacked_provisional_designation, JDlist[:n], HPlist)

OperationalError: near "objects_by_jdhp": syntax error

### Demonstrate code to query Healpix by desig ...

In [279]:
# sql query ...
result = sql.query_JDHP_by_name(conn, MSCs[0].primary_unpacked_provisional_designation)

# Examine the characteristics of the output 
print('Characteristics of query results ...' , type(result) , len(result) )
print(result[0])

# Compare the output to the input to verify is the same ... 
#if np.all(MSCs[0].sector_coeffs[ int(key0.split("_")[1]) ] == result[key0] ) : print( 'In == Out !!')

Characteristics of query results ... <class 'tuple'> 2
b'@;%\x00\x00\x00\x00\x00'


# --------- PreCalc ---------------------


 - A core part of the PreCalc routine is the generation of HP-positions on a nightly-basis
 - These calculations require that the positions of the observatory are provided as inputs: I.e. I want to completely separate the obs-posn stuff from the cheby evaluation calculations
 - For the sake of these demonstrations, I am going to generate a set of standardized input observatory-positions (using a silly ~circular approximation) and then provide those as inputs. 
     

In [198]:
# Create a sample of observatory-positions as inputs 
# - Note that the NBody code uses BARYCENTRIC EQUATORIAL coords
# - So the Cheby's need to work-in/evaluate-to equatorial coords
# - Hence the observatory-positions will need to be in equatorial coords
nSample = int(1e2) 
evaluation_times = np.sort(2440000. + 999.*np.random.rand(nSample) )
r = 1.0 
XYZobservatory_equatorial = np.array( [r*np.cos(evaluation_times), 
                                        r*np.sin(evaluation_times) , 
                                        0.0001*r*np.cos(evaluation_times)])

In [200]:
# Repeat the demo from the "Demonstrate_Orbital_Chebyshev_Functionality" notebook
# - How to calculate HP ...
MSCs = create_sample_cheby_dict()
HPs=MSCs[0].generate_HP( evaluation_times , 
                XYZobservatory_equatorial, 
                APPROX = True )
# N.B. final shape = (3,len(times_TDB))
HPs.shape , evaluation_times.shape


INIT MSC_Loader...


((100,), (100,))

In [ ]:
## Demonstrate the main PreCalc *upsert* func
 - This 
    
    (i) uses the sql.upsert_MSC() function from above
    (ii) uses the .generate_HP*( function)
    (iii) uses the sql.upsert